In [1]:
# Script to post-process high-resolution WRF model output.

# Major tasks include computing the following for selected variables:
#   1. domain-averages to produce time series
#   2. vertical integrals
#   3. pressure-level vertical interpolation

In [1]:
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, disable_xarray, ALL_TIMES
from read_wrf_ideal import *
from post_proc_functions import *
import os

disable_xarray()

In [ ]:
# Basic 2D variables
do_2d_vars = False

# Special 2D variables
do_2d_special = True

# 2D ACRE variables
do_acre = True

# Cloud classification
# do_pclass = False

In [ ]:
test_process = "ctl"

#### Directories and model output specs

datdir = "/glade/derecho/scratch/ruppert/wrf-ideal/smalldom/"
wrfdir = "/glade/campaign/univ/uokl0049/ideal/largedom2/"+test_process+"/"
outdir = wrfdir+"post_proc/"
os.makedirs(outdir, exist_ok=True)

# # Get WRF file list, dimensions
wrftag = "wrfout_d01"
wrffiles = get_wrf_file_list(wrfdir, wrftag)
lat, lon, nx1, nx2, nz = wrf_dims(wrffiles[0])
nfiles = len(wrffiles)

# New vertical dimension for pressure levels
dp = 25 # hPa
pres = np.arange(1000, 25, -dp)
nznew = len(pres)

# Get variable lists
vars2d = var_list_2d()
vars3d = var_list_3d()

In [ ]:
# Use CDO mergetime to generate files of basic 2D variables

if do_2d_vars:

    for ivar in vars2d[0:1]:

        varname_str = ivar.upper()
        operation_str1 = 'cdo mergetime'
        operation_str2 = ' -selname,'+varname_str+' '
        out_file = ' '+outdir+varname_str+'.nc'

        # Create string array of file-specific commands
        cdo_line = [operation_str1]
        for ifil in wrffiles[0:3]:
            cdo_line.append(operation_str2+ifil)
        # Then join them into one string
        cdo_line_merged = " ".join(cdo_line)

        # Run CDO command
        process = subprocess.Popen([cdo_line_merged+out_file], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        # lines = process.stdout.readlines()
        # lines = process.stderr.readlines()
        # for iline in lines:
        #     print(iline)

In [ ]:
# Use CDO subtraction to generate ACRE

if do_acre:

    # Calculate the longwave ACRE

    operation_str = 'cdo sub '+outdir+'LWUPT.nc '+outdir+'LWDNT.nc lw_t.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    operation_str = 'cdo sub '+outdir+'LWUPB.nc '+outdir+'LWDNB.nc lw_b.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    operation_str = 'cdo sub '+outdir+'lw_b.nc '+outdir+'lw_t.nc lw_net.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    operation_str = 'cdo sub '+outdir+'LWUPTC.nc '+outdir+'LWDNTC.nc lw_tC.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    operation_str = 'cdo sub '+outdir+'LWUPBC.nc '+outdir+'LWDNBC.nc lw_bC.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    operation_str = 'cdo sub '+outdir+'lw_bC.nc '+outdir+'lw_tC.nc lw_netC.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    operation_str = 'cdo sub '+outdir+'lw_net.nc '+outdir+'lw_netC.nc LWacre.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    # Calculate the shortwave ACRE

    operation_str = 'cdo sub '+outdir+'SWUPT.nc '+outdir+'SWDNT.nc sw_t.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    operation_str = 'cdo sub '+outdir+'SWUPB.nc '+outdir+'SWDNB.nc sw_b.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    operation_str = 'cdo sub '+outdir+'sw_b.nc '+outdir+'sw_t.nc sw_net.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    operation_str = 'cdo sub '+outdir+'SWUPTC.nc '+outdir+'SWDNTC.nc sw_tC.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    operation_str = 'cdo sub '+outdir+'SWUPBC.nc '+outdir+'SWDNBC.nc sw_bC.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    operation_str = 'cdo sub '+outdir+'sw_bC.nc '+outdir+'sw_tC.nc sw_netC.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    operation_str = 'cdo sub '+outdir+'sw_net.nc '+outdir+'sw_netC.nc SWacre.nc'
    process = subprocess.Popen(operation_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    # lines = process.stdout.readlines()
    # lines = process.stderr.readlines()
    # for iline in lines:
    #     print(iline)

In [ ]:
def write_ncfile(file_out, var, var_name, description, units, dims_set): #, dim_names
    ncfile = Dataset(file_out,mode='w', clobber=True)
    # Add dimensions to file
    for idim in range(len(dims_set)):
        dim = ncfile.createDimension(dims_set[0][idim], dims_set[1][idim]) # unlimited axis (can be appended to).

    writevar = ncfile.createVariable(var_name, np.single, dims_set[0]) #dim_names[ivar])
    writevar.units = units
    writevar.description = description
    writevar[...] = var

    ncfile.close()

    print("Done writing!")
    
    return None

In [8]:
if do_2d_special:

    # Read in variable from WRF files
    # for ifile in range(nfiles):
    for ifile in range(3):

        # Read in hydrostatic pressure to get dp for integral
        p_hyd = wrf_var_read(wrffiles[ifile],'P_HYD') # Pa
        # p_hyd = np.ma.masked_where((p_hyd < 100e2), p_hyd, copy=False) # Mask out levels above 100 hPa
        dp = np.gradient(p_hyd, axis=0, edge_order=1) # [Pa] Uses second order centered differencing

        # pclass
        var = wrf_pclass(wrffiles[ifile], dp)
        if ifile == 0:
            pclass_all = var
        else:
            pclass_all = np.concatenate((pclass_all, var), axis=0)

        # rain
        var = wrf_var_read(wrffiles[ifile], 'RAINNC')
        if ifile == 0:
            rainnc_all = var
        else:
            rainnc_all = np.concatenate((rainnc_all, var), axis=0)

        # pw
        var = wrf_pw(wrffiles[ifile], p_hyd, dp)
        if ifile == 0:
            pw_all = var
        else:
            pw_all = np.concatenate((pw_all, var), axis=0)

        # pw_sat
        var = wrf_pw_sat(wrffiles[ifile], p_hyd, dp)
        if ifile == 0:
            pw_sat_all = var
        else:
            pw_sat_all = np.concatenate((pw_sat_all, var), axis=0)


: 

In [ ]:
# Read in variable from WRF files
# for ifile in range(len(wrffiles)):
for ifile in range(2):
    wrfdset = Dataset(wrffiles[ifile])
    var = wrfdset.variables[ivar.upper()][:,:,:]
    wrfdset.close()
    if ifile == 0:
        var_all = var
    else:
        var_all = np.concatenate((var_all, var), axis=0)